# 11. Magentic Workflow Agent Workshop

이 노트북에서는 Microsoft Agent Framework의 Magentic 워크플로우를 사용하여 복합 에이전트 시스템을 구축하는 방법을 학습합니다.

## 학습 목표
- Magentic 워크플로우의 개념과 구조 이해
- 다중 에이전트 오케스트레이션 구현
- 워크플로우를 에이전트로 래핑하여 재사용하는 방법
- 스트리밍 이벤트 처리 및 콜백 구현

## 시나리오
연구자와 코더 에이전트가 협력하여 머신러닝 모델의 에너지 효율성을 분석하는 워크플로우를 구축합니다.

## 1. 필수 라이브러리 및 모듈 임포트

In [4]:
# Microsoft Agent Framework의 Magentic 워크플로우 관련 모듈들
import asyncio
import logging
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv(override=True)

from agent_framework import (
    ChatAgent,
    HostedCodeInterpreterTool,
    MagenticAgentDeltaEvent,
    MagenticAgentMessageEvent,
    MagenticBuilder,
    MagenticFinalResultEvent,
    MagenticOrchestratorMessageEvent,
    WorkflowOutputEvent,
)
# Azure OpenAI 클라이언트 사용
from agent_framework.azure import AzureOpenAIChatClient

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("라이브러리 임포트 완료!")

라이브러리 임포트 완료!


## 2. Azure OpenAI 설정

Magentic 워크플로우에서 사용할 Azure OpenAI 클라이언트를 설정합니다.
- `AzureOpenAIChatClient`: Azure OpenAI 서비스를 통한 채팅 및 분석 작업용
- 환경변수를 통한 Azure OpenAI 연결 설정

In [5]:
# 환경 변수 확인
print("환경 변수 확인:")
env_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_KEY", 
    "AZURE_OPENAI_API_VERSION",
    "AZURE_OPENAI_CHAT_DEPLOYMENT_NAME",
    "OPENAI_API_KEY"
]

for var in env_vars:
    value = os.getenv(var)
    if value:
        # API 키는 부분적으로만 표시
        if "API_KEY" in var:
            print(f"✅ {var}: {value[:8]}...")
        else:
            print(f"✅ {var}: {value}")
    else:
        print(f"❌ {var}: 설정되지 않음")

print("\n💡 환경 변수가 설정되지 않은 경우, Azure CLI 인증을 사용합니다.")

환경 변수 확인:
✅ AZURE_OPENAI_ENDPOINT: https://aistudioaiservices343274518006.openai.azure.com
✅ AZURE_OPENAI_API_KEY: 6hBhIb88...
❌ AZURE_OPENAI_API_VERSION: 설정되지 않음
✅ AZURE_OPENAI_CHAT_DEPLOYMENT_NAME: gpt-4.1
❌ OPENAI_API_KEY: 설정되지 않음

💡 환경 변수가 설정되지 않은 경우, Azure CLI 인증을 사용합니다.


### Azure OpenAI 설정 예제

Azure OpenAI를 사용하기 위해 다음 환경변수들을 `.env` 파일에 설정하세요:

```bash
# .env 파일 예제
AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/
AZURE_OPENAI_API_KEY=your-api-key-here
AZURE_OPENAI_API_VERSION=2024-06-01
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME=gpt-4o
```

## 3. 에이전트 생성

두 개의 전문 에이전트를 생성합니다:
1. **Researcher Agent**: 연구 및 정보 수집 전문
2. **Coder Agent**: 코드 작성 및 데이터 분석 전문

In [6]:
# Azure OpenAI 클라이언트 설정 함수
def create_azure_chat_client():
    """Azure OpenAI 클라이언트 생성"""
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    api_key = os.getenv("AZURE_OPENAI_API_KEY")
    api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-06-01")
    deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    
    if endpoint and api_key and deployment_name:
        return AzureOpenAIChatClient(
            endpoint=endpoint,
            api_key=api_key,
            api_version=api_version,
            deployment_name=deployment_name
        )
    else:
        print("⚠️  Azure OpenAI 설정이 불완전합니다. OpenAI 클라이언트를 사용합니다.")
        # Fallback to OpenAI if Azure settings are not complete
        from agent_framework.openai import OpenAIChatClient
        return OpenAIChatClient(model_id="gpt-4o")

# 연구자 에이전트 생성
researcher_agent = ChatAgent(
    name="ResearcherAgent",
    description="Specialist in research and information gathering",
    instructions=(
        "You are a Researcher. You find information without additional "
        "computation or quantitative analysis. Focus on gathering factual "
        "information, research papers, and established knowledge."
    ),
    # Azure OpenAI 클라이언트 사용
    chat_client=create_azure_chat_client(),
)

print("✅ ResearcherAgent 생성 완료")

# 코더 에이전트 생성  
coder_agent = ChatAgent(
    name="CoderAgent",
    description="A helpful assistant that writes and executes code to process and analyze data.",
    instructions=(
        "You solve questions using code. Please provide detailed analysis "
        "and computation process. Write clean, well-documented code."
    ),
    chat_client=create_azure_chat_client(),
    tools=HostedCodeInterpreterTool(),  # 코드 실행 도구
)

print("✅ CoderAgent 생성 완료")
print("두 에이전트가 성공적으로 생성되었습니다!")

✅ ResearcherAgent 생성 완료
✅ CoderAgent 생성 완료
두 에이전트가 성공적으로 생성되었습니다!


## 4. Magentic 워크플로우 구축

MagenticBuilder를 사용하여 다중 에이전트 워크플로우를 구성합니다:
- **Participants**: 워크플로우에 참여할 에이전트들 정의
- **Manager**: 에이전트 간 조정 및 관리 설정

In [7]:
print("🔧 Magentic 워크플로우 구축 중...")

# MagenticBuilder를 사용하여 워크플로우 구성
workflow = (
    MagenticBuilder()
    .participants(researcher=researcher_agent, coder=coder_agent)
    .with_standard_manager(
        chat_client=create_azure_chat_client(),  # Azure OpenAI 클라이언트 사용
        max_round_count=10,     # 최대 대화 라운드 수
        max_stall_count=3,      # 진행 정체 최대 허용 횟수
        max_reset_count=2,      # 워크플로우 재시작 최대 횟수
    )
    .build()
)

print("✅ Magentic 워크플로우가 성공적으로 구축되었습니다!")
print(f"참여 에이전트: {len([researcher_agent, coder_agent])}개")
print("워크플로우 매니저 설정 완료")

🔧 Magentic 워크플로우 구축 중...
✅ Magentic 워크플로우가 성공적으로 구축되었습니다!
참여 에이전트: 2개
워크플로우 매니저 설정 완료


## 5. 분석 작업 정의

머신러닝 모델의 에너지 효율성을 비교 분석하는 복합적인 작업을 정의합니다.

In [8]:
# 분석 작업 정의 - 머신러닝 모델 에너지 효율성 비교
task = (
    "I am preparing a report on the energy efficiency of different machine learning model architectures. "
    "Compare the estimated training and inference energy consumption of ResNet-50, BERT-base, and GPT-2 "
    "on standard datasets (e.g., ImageNet for ResNet, GLUE for BERT, WebText for GPT-2). "
    "Then, estimate the CO2 emissions associated with each, assuming training on an Azure Standard_NC6s_v3 "
    "VM for 24 hours. Provide tables for clarity, and recommend the most energy-efficient model "
    "per task type (image classification, text classification, and text generation)."
)

print("📋 분석 작업:")
print("=" * 60)
print(task)
print("=" * 60)
print("\n이 작업은 다음과 같은 단계로 수행됩니다:")
print("1. 연구자 에이전트: 각 모델의 에너지 소비 데이터 조사")
print("2. 코더 에이전트: 수집된 데이터를 기반으로 계산 및 비교 분석")
print("3. 협업: 최종 보고서 및 권장사항 작성")

📋 분석 작업:
I am preparing a report on the energy efficiency of different machine learning model architectures. Compare the estimated training and inference energy consumption of ResNet-50, BERT-base, and GPT-2 on standard datasets (e.g., ImageNet for ResNet, GLUE for BERT, WebText for GPT-2). Then, estimate the CO2 emissions associated with each, assuming training on an Azure Standard_NC6s_v3 VM for 24 hours. Provide tables for clarity, and recommend the most energy-efficient model per task type (image classification, text classification, and text generation).

이 작업은 다음과 같은 단계로 수행됩니다:
1. 연구자 에이전트: 각 모델의 에너지 소비 데이터 조사
2. 코더 에이전트: 수집된 데이터를 기반으로 계산 및 비교 분석
3. 협업: 최종 보고서 및 권장사항 작성


## 6. 워크플로우 실행 (스트리밍)

워크플로우를 실행하고 실시간 스트리밍 이벤트를 처리합니다. 
각 에이전트의 작업 진행 상황과 결과를 실시간으로 확인할 수 있습니다.

In [9]:
async def run_workflow_with_streaming():
    """워크플로우를 실행하고 스트리밍 이벤트 처리"""
    
    print("🚀 워크플로우 실행 시작...")
    print("=" * 70)
    
    try:
        # 스트리밍 상태 관리 변수
        last_stream_agent_id = None
        stream_line_open = False
        final_output = None
        
        # 워크플로우 실행 및 스트리밍 이벤트 처리
        async for event in workflow.run_stream(task):
            
            # 오케스트레이터 메시지 이벤트 처리
            if isinstance(event, MagenticOrchestratorMessageEvent):
                event_text = getattr(event.message, 'text', '')
                print(f"\n[ORCHESTRATOR:{event.kind}]")
                print(f"{event_text}")
                print("-" * 50)
            
            # 에이전트 델타 이벤트 (실시간 응답 스트리밍)
            elif isinstance(event, MagenticAgentDeltaEvent):
                if last_stream_agent_id != event.agent_id or not stream_line_open:
                    if stream_line_open:
                        print()
                    print(f"\n[STREAMING:{event.agent_id}]: ", end="", flush=True)
                    last_stream_agent_id = event.agent_id
                    stream_line_open = True
                
                if event.text:
                    print(event.text, end="", flush=True)
            
            # 에이전트 메시지 완료 이벤트
            elif isinstance(event, MagenticAgentMessageEvent):
                if stream_line_open:
                    print(" (완료)")
                    stream_line_open = False
                    print()
                
                msg = event.message
                if msg is not None:
                    response_text = (msg.text or "").replace("\n", " ")[:200] + "..."
                    print(f"\n[AGENT:{event.agent_id}] {msg.role.value}")
                    print(f"{response_text}")
                    print("-" * 50)
            
            # 최종 결과 이벤트
            elif isinstance(event, MagenticFinalResultEvent):
                print("\n" + "=" * 50)
                print("🎉 최종 결과:")
                print("=" * 50)
                if event.message is not None:
                    print(event.message.text)
                print("=" * 50)
            
            # 워크플로우 출력 이벤트
            elif isinstance(event, WorkflowOutputEvent):
                final_output = str(event.data) if event.data is not None else None
        
        # 스트리밍 정리
        if stream_line_open:
            print()
            stream_line_open = False
        
        if final_output is not None:
            print(f"\n✅ 워크플로우 완료!")
            print(f"결과: {final_output[:500]}..." if len(final_output) > 500 else final_output)
        
        return final_output
    
    except Exception as e:
        print(f"❌ 워크플로우 실행 실패: {e}")
        return None

# 데모 실행을 위한 간단한 버전 (실제 API 키가 있을 때만 실행)
print("워크플로우 스트리밍 함수가 정의되었습니다.")
print("실제 실행하려면 다음 셀을 실행하세요 (OpenAI API 키 필요).")

워크플로우 스트리밍 함수가 정의되었습니다.
실제 실행하려면 다음 셀을 실행하세요 (OpenAI API 키 필요).


In [10]:
# 워크플로우 실행 (Azure OpenAI 또는 OpenAI API 키가 설정된 경우에만 실행)
# 주석을 해제하고 실행하세요

# # Azure OpenAI 또는 OpenAI 설정 확인
# azure_configured = all([
#     os.getenv("AZURE_OPENAI_ENDPOINT"),
#     os.getenv("AZURE_OPENAI_API_KEY"),
#     os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
# ])
# openai_configured = bool(os.getenv("OPENAI_API_KEY"))

# if azure_configured or openai_configured:
#     result = await run_workflow_with_streaming()
#     print(f"\n워크플로우 실행 결과: {result}")
# else:
#     print("⚠️  워크플로우 실행을 위해서는 Azure OpenAI 또는 OpenAI API 설정이 필요합니다.")
#     print("환경변수를 설정한 후 다시 실행해주세요.")

print("💡 워크플로우 실행 준비 완료!")
print("실제 실행을 위해서는 위의 주석을 해제하고 Azure OpenAI 또는 OpenAI API를 설정하세요.")

💡 워크플로우 실행 준비 완료!
실제 실행을 위해서는 위의 주석을 해제하고 Azure OpenAI 또는 OpenAI API를 설정하세요.


## 7. 워크플로우를 에이전트로 래핑

Magentic의 강력한 기능 중 하나는 전체 워크플로우를 단일 에이전트로 래핑하여 다른 시스템에서 재사용할 수 있다는 점입니다.

In [11]:
async def demo_workflow_as_agent():
    """워크플로우를 에이전트로 래핑하여 사용하는 예제"""
    
    print("🔄 워크플로우를 에이전트로 래핑...")
    
    # 워크플로우를 단일 에이전트로 래핑
    workflow_agent = workflow.as_agent(name="MagenticWorkflowAgent")
    
    print("✅ 워크플로우 에이전트 생성 완료!")
    print(f"에이전트 이름: {workflow_agent}")
    
    # 간단한 테스트 작업
    simple_task = "Summarize the key factors that affect machine learning model energy efficiency."
    
    print(f"\n📝 테스트 작업: {simple_task}")
    
    try:
        # 래핑된 에이전트로 작업 실행
        agent_result = await workflow_agent.run(simple_task)
        
        if agent_result.messages:
            print("\n" + "=" * 60)
            print("📋 에이전트 실행 결과 (대화 기록)")
            print("=" * 60)
            
            for i, msg in enumerate(agent_result.messages, start=1):
                role_value = getattr(msg.role, "value", msg.role)
                speaker = msg.author_name or role_value
                print(f"\n{'-' * 50}")
                print(f"{i:02d} [{speaker}]")
                print(f"{msg.text}")
        
        return agent_result
    
    except Exception as e:
        print(f"❌ 에이전트 실행 실패: {e}")
        return None

# 워크플로우 에이전트 래핑 함수 정의 완료
print("📦 워크플로우 에이전트 래핑 함수가 정의되었습니다.")
print("실제 실행하려면 다음 셀을 실행하세요.")

📦 워크플로우 에이전트 래핑 함수가 정의되었습니다.
실제 실행하려면 다음 셀을 실행하세요.


In [12]:
# 워크플로우 에이전트 데모 실행 (Azure OpenAI 또는 OpenAI API 키가 필요)
# 주석을 해제하고 실행하세요

# # Azure OpenAI 또는 OpenAI 설정 확인
# azure_configured = all([
#     os.getenv("AZURE_OPENAI_ENDPOINT"),
#     os.getenv("AZURE_OPENAI_API_KEY"),
#     os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
# ])
# openai_configured = bool(os.getenv("OPENAI_API_KEY"))

# if azure_configured or openai_configured:
#     agent_result = await demo_workflow_as_agent()
#     if agent_result:
#         print("\n🎉 워크플로우 에이전트 실행 성공!")
# else:
#     print("⚠️  워크플로우 에이전트 실행을 위해서는 Azure OpenAI 또는 OpenAI API 설정이 필요합니다.")

print("🎯 워크플로우 에이전트 데모 준비 완료!")
print("이제 복잡한 다중 에이전트 워크플로우를 단일 에이전트처럼 사용할 수 있습니다.")

🎯 워크플로우 에이전트 데모 준비 완료!
이제 복잡한 다중 에이전트 워크플로우를 단일 에이전트처럼 사용할 수 있습니다.


## 8. 고급 기능 및 설정

Magentic 워크플로우의 고급 기능들을 살펴보겠습니다.

In [13]:
# Magentic 워크플로우의 고급 설정들

print("🔧 Magentic 워크플로우 고급 설정 가이드")
print("=" * 60)

# 1. 매니저 설정 매개변수들
print("\n1. 워크플로우 매니저 설정:")
print("   - max_round_count: 최대 대화 라운드 (기본값: 10)")
print("   - max_stall_count: 진행 정체 허용 횟수 (기본값: 3)")
print("   - max_reset_count: 워크플로우 재시작 횟수 (기본값: 2)")

# 2. 이벤트 타입들
print("\n2. 스트리밍 이벤트 타입들:")
event_types = {
    "MagenticOrchestratorMessageEvent": "오케스트레이터의 조정 메시지",
    "MagenticAgentDeltaEvent": "에이전트의 실시간 응답 스트림",
    "MagenticAgentMessageEvent": "에이전트의 완성된 메시지",
    "MagenticFinalResultEvent": "워크플로우의 최종 결과",
    "WorkflowOutputEvent": "워크플로우 출력 데이터"
}

for event_type, description in event_types.items():
    print(f"   - {event_type}: {description}")

# 3. 에이전트 도구들
print("\n3. 사용 가능한 에이전트 도구들:")
tools_info = {
    "HostedCodeInterpreterTool": "코드 실행 및 분석",
    "WebSearchTool": "웹 검색 (모델 의존적)",
    "FileAccessTool": "파일 읽기/쓰기 작업",
    "DatabaseTool": "데이터베이스 연동"
}

for tool, description in tools_info.items():
    print(f"   - {tool}: {description}")

print("\n4. 모범 사례:")
print("   ✅ 에이전트별로 명확한 역할과 지시사항 정의")
print("   ✅ 적절한 모델 선택 (검색: gpt-4o-search, 코드: gpt-4o)")
print("   ✅ 스트리밍 이벤트 적절히 처리하여 사용자 경험 향상")
print("   ✅ 에러 처리 및 재시도 로직 구현")
print("   ✅ 워크플로우를 에이전트로 래핑하여 재사용성 향상")

🔧 Magentic 워크플로우 고급 설정 가이드

1. 워크플로우 매니저 설정:
   - max_round_count: 최대 대화 라운드 (기본값: 10)
   - max_stall_count: 진행 정체 허용 횟수 (기본값: 3)
   - max_reset_count: 워크플로우 재시작 횟수 (기본값: 2)

2. 스트리밍 이벤트 타입들:
   - MagenticOrchestratorMessageEvent: 오케스트레이터의 조정 메시지
   - MagenticAgentDeltaEvent: 에이전트의 실시간 응답 스트림
   - MagenticAgentMessageEvent: 에이전트의 완성된 메시지
   - MagenticFinalResultEvent: 워크플로우의 최종 결과
   - WorkflowOutputEvent: 워크플로우 출력 데이터

3. 사용 가능한 에이전트 도구들:
   - HostedCodeInterpreterTool: 코드 실행 및 분석
   - WebSearchTool: 웹 검색 (모델 의존적)
   - FileAccessTool: 파일 읽기/쓰기 작업
   - DatabaseTool: 데이터베이스 연동

4. 모범 사례:
   ✅ 에이전트별로 명확한 역할과 지시사항 정의
   ✅ 적절한 모델 선택 (검색: gpt-4o-search, 코드: gpt-4o)
   ✅ 스트리밍 이벤트 적절히 처리하여 사용자 경험 향상
   ✅ 에러 처리 및 재시도 로직 구현
   ✅ 워크플로우를 에이전트로 래핑하여 재사용성 향상


## 9. 실습 과제 및 확장

다음과 같은 실습을 통해 Magentic 워크플로우를 더 깊이 이해해보세요.

In [14]:
print("🎯 Magentic 워크플로우 실습 과제")
print("=" * 50)

exercises = {
    "초급": [
        "다른 도메인의 전문 에이전트 추가 (예: 디자이너, 마케터)",
        "워크플로우 매니저 설정 값 조정하여 성능 비교",
        "다른 OpenAI 모델들로 에이전트 구성해보기"
    ],
    "중급": [
        "커스텀 도구를 가진 에이전트 생성",
        "조건부 워크플로우 로직 구현",
        "에러 처리 및 재시도 메커니즘 강화"
    ],
    "고급": [
        "동적으로 에이전트를 추가/제거하는 워크플로우",
        "외부 API와 연동하는 에이전트 개발",
        "복수의 워크플로우를 연계하는 메타 워크플로우"
    ]
}

for level, tasks in exercises.items():
    print(f"\n{level} 실습:")
    for i, task in enumerate(tasks, 1):
        print(f"   {i}. {task}")

print("\n" + "=" * 50)
print("🚀 확장 아이디어:")
print("   • 실시간 협업 플랫폼 구축")
print("   • 자동화된 리서치 파이프라인")
print("   • 코드 리뷰 및 개선 워크플로우")
print("   • 멀티모달 에이전트 (텍스트 + 이미지 + 음성)")
print("   • 장기 메모리를 가진 지속적 학습 에이전트")

print("\n💡 다음 단계:")
print("   1. OpenAI API 키 설정 후 실제 워크플로우 실행")
print("   2. 다양한 에이전트 조합으로 실험")
print("   3. 실제 비즈니스 문제에 적용해보기")
print("   4. 성능 모니터링 및 최적화")

🎯 Magentic 워크플로우 실습 과제

초급 실습:
   1. 다른 도메인의 전문 에이전트 추가 (예: 디자이너, 마케터)
   2. 워크플로우 매니저 설정 값 조정하여 성능 비교
   3. 다른 OpenAI 모델들로 에이전트 구성해보기

중급 실습:
   1. 커스텀 도구를 가진 에이전트 생성
   2. 조건부 워크플로우 로직 구현
   3. 에러 처리 및 재시도 메커니즘 강화

고급 실습:
   1. 동적으로 에이전트를 추가/제거하는 워크플로우
   2. 외부 API와 연동하는 에이전트 개발
   3. 복수의 워크플로우를 연계하는 메타 워크플로우

🚀 확장 아이디어:
   • 실시간 협업 플랫폼 구축
   • 자동화된 리서치 파이프라인
   • 코드 리뷰 및 개선 워크플로우
   • 멀티모달 에이전트 (텍스트 + 이미지 + 음성)
   • 장기 메모리를 가진 지속적 학습 에이전트

💡 다음 단계:
   1. OpenAI API 키 설정 후 실제 워크플로우 실행
   2. 다양한 에이전트 조합으로 실험
   3. 실제 비즈니스 문제에 적용해보기
   4. 성능 모니터링 및 최적화


## 요약 및 핵심 포인트

### Magentic 워크플로우의 핵심 개념

1. **다중 에이전트 오케스트레이션**: 여러 전문 에이전트가 협력하여 복잡한 작업 수행
2. **스트리밍 이벤트**: 실시간으로 워크플로우 진행 상황 모니터링
3. **에이전트 래핑**: 워크플로우 전체를 단일 에이전트로 재사용 가능
4. **유연한 설정**: 매니저 설정을 통한 워크플로우 동작 제어

### 주요 장점
- ✅ **모듈성**: 각 에이전트가 독립적인 전문 영역 담당
- ✅ **확장성**: 새로운 에이전트 쉽게 추가 가능
- ✅ **재사용성**: as_agent()를 통한 워크플로우 재활용
- ✅ **투명성**: 스트리밍을 통한 실시간 진행 상황 파악

### 활용 사례
- 📊 **연구 및 분석**: 정보 수집 + 데이터 분석 + 보고서 작성
- 🤖 **소프트웨어 개발**: 요구사항 분석 + 코드 생성 + 테스트 + 리뷰
- 📈 **비즈니스 자동화**: 데이터 수집 + 분석 + 의사결정 지원
- 🎓 **교육 도구**: 질문 분석 + 학습자료 검색 + 맞춤형 설명 생성

이 워크샵을 통해 Microsoft Agent Framework의 Magentic 워크플로우를 활용한 지능형 에이전트 시스템 구축 방법을 학습했습니다!